# Description
This kernel performs inference for [PANDA concat tile pooling starter](https://www.kaggle.com/iafoss/panda-concat-fast-ai-starter) kernel with use of multiple models and 8 fold TTA. Check it for more training details. The image preprocessing pipline is provided [here](https://www.kaggle.com/iafoss/panda-16x128x128-tiles).

In [ ]:
import cv2
from tqdm import tqdm_notebook as tqdm
import fastai
from fastai.vision import *
import os
#from mish_activation import *
import warnings
warnings.filterwarnings("ignore")
import skimage.io
import numpy as np
import pandas as pd
#sys.path.insert(0, '../input/semisupervised-imagenet-models/semi-supervised-ImageNet1K-models-master/')
#from hubconf import *
import sys
sys.path.append('/kaggle/input/panda-utils-cleaned/') #sys.path.append('/kaggle/input/panda-utils/')
from model.model_config import ModelConfig
from model.model_utils import load_models_from_dir

In [ ]:
DATA = '../input/prostate-cancer-grade-assessment/test_images'
TEST = '../input/prostate-cancer-grade-assessment/test.csv'
SAMPLE = '../input/prostate-cancer-grade-assessment/sample_submission.csv'

bs = 2
nworkers = 2
coef = [0.5, 1.5, 2.5, 3.5, 4.5]

# Change these settings

In [ ]:
model_dirs = ['../input/jj20200629', '../input/jj20200701/', '../input/jj20200703/', '../input/jj20200706']
model_weights = np.array([3,4,2,1])
use_models = [[True,True,False,False], [True,True,False,False], [True,True,False,False], [True,False,False,False]]
tile_size = [384,256,256,299]
level=[1,1,1,1]
mosaic_grid = [(4,6),(6,6),(6,6),(5,5)]
tissue_th = (0.2,0.7)
seed = 123
sampling_method=['skeleton','skeleton','skeleton','skeleton']

The next cell will load your models and read additional settings from the model config.

In [ ]:
# load config
config = [ModelConfig.fromDir(model_dir) for model_dir in model_dirs]
# load models
models = [load_models_from_dir(model_dir, tile_list_input=False) for model_dir in model_dirs]
cms = [[np.load(os.path.join(model_dir, f'cm_fold-{i}.npy')) for i in range(len(models[0]))] for model_dir in model_dirs]

models = [[model for model, use_model in zip(_models,_use_models) if use_model] for _models,_use_models in zip(models,use_models)]
cms = [[cm for cm, use_model in zip(_cms,_use_models) if use_model] for _cms,_use_models in zip(cms,use_models)]

sz = [_config.getField('sz') for _config in config]
N = [_config.getField('N') for _config in config]
is_ordinal = [_config.getMetaField('is_ordinal')==True for _config in config]
model_name = [_config.getField('model_name') for _config in config]
regr = ["regr" in _model_name for _model_name in model_name]
mean = [torch.tensor(np.array(_config.getField('mean')).astype(np.float32)) for _config in config]
std = [torch.tensor(np.array(_config.getField('std')).astype(np.float32)) for _config in config]
#assert np.multiply(*mosaic_grid) == N, "mosaic grid is different from the N read from the config"

for i, _models in enumerate(models):
    print("Loaded {0} {1} models".format(len(_models), model_name[i]))
    print(f'Mean {mean[i]} and std {std[i]}')
    print(f'regression = {regr[i]}')
    print(f'ordinal regression = {is_ordinal[i]}')

In [ ]:
print(sz, tile_size)

In [ ]:
N

## slide_preprocessing.py

In [ ]:
import pandas as pd
import numpy as np

from skimage.io import MultiImage
from skimage.morphology import skeletonize
#import maskslic as seg

import cv2
import matplotlib.pyplot as plt

from pathlib import Path

import warnings

VALID_SLIDE_EXTENSIONS = {'.tiff', '.mrmx', '.svs'}


# ~~~~~~~~~~~~ Helper functions ~~~~~~~~~~~~
def generateMetaDF(data_dir, meta_fn:str='train.csv'):
    '''
        Makes a pandas.DataFrame of paths out of a directory including slides. Drop the `train.csv` in `data_dir`
        and the script will also merge any meta data from there on `image_id` key.
    '''
    
    
    all_files = [path.resolve() for path in Path(data_dir).rglob("*.*")]
    slide_paths = [path for path in all_files if path.suffix in VALID_SLIDE_EXTENSIONS]
    
    if len(slide_paths)==0:
        raise ValueError('No slides in `data_dir`=%s'%data_dir)
    
    data_df = pd.DataFrame({'slide_path':slide_paths})
    data_df['image_id'] = data_df.slide_path.apply(lambda x: x.stem)
    
    slides = data_df[~data_df.image_id.str.contains("mask")]
    masks = data_df[data_df.image_id.str.contains("mask")]
    masks['image_id'] = masks.image_id.str.replace("_mask", "")
    masks.columns = ['mask_path', 'image_id']
    
    data_df = slides.merge(masks, on='image_id', how='left')
    data_df['slide_path'] = data_df.slide_path.apply(lambda x: str(x) if not pd.isna(x) else None)
    data_df['mask_path'] = data_df.mask_path.apply(lambda x: str(x) if not pd.isna(x) else None)
    

    ## Merge metadata
    meta_csv = [file for file in all_files if file.name==meta_fn]
    if meta_csv:
        meta_df = pd.read_csv(str(meta_csv[0]))
        data_df = data_df.merge(meta_df, on='image_id')
    
    return data_df

def tileClassification(tile, provider:str):
    '''
        Returns the cancer class of a tile based on majority vote of the tile's annotated pixels
            0: background (non tissue) or unknown
            1: benign tissue (stroma and epithelium combined)
            2: cancerous tissue (stroma and epithelium combined)
    '''

    if provider == "karolinska":
        '''
        Karolinska: Regions are labelled. Valid values are:
            0: background (non tissue) or unknown
            1: benign tissue (stroma and epithelium combined)
            2: cancerous tissue (stroma and epithelium combined)
        '''
        classes = {0:0, 1:1, 2:2}    
        
    elif provider == "radboud":
        ''' 
        Radboud: Prostate glands are individually labelled. Valid values are:
            0: background (non tissue) or unknown
            1: stroma (connective tissue, non-epithelium tissue)
            2: healthy (benign) epithelium
            3: cancerous epithelium (Gleason 3)
            4: cancerous epithelium (Gleason 4)
            5: cancerous epithelium (Gleason 5)
        '''
        classes = {0:0, 1:1, 2:1, 3:2, 4:2, 5:2}
    
    tile = np.int32(tile)
    counts = np.bincount(tile.reshape(-1,1)[:,0])

    ## If only background, accept background (class=0) as the annotation
    if len(counts)==1:
        return 0

    ## Otherwise take the second most common pixel as the annotation
    max_annotation = np.argmax(counts[1:])       
    
    return classes[max_annotation+1]
    
def getTopLeftCorners(dims, tile_size:int):
    ''' Make a map of the tiles' locations '''
    
    cols, rows = np.ceil(dims/tile_size).astype(int)

    ## M
    top_left_corners = []
    for i in range(cols):
        for j in range(rows):
            top_left_corners.append( (i*tile_size, j*tile_size) )
            
    return np.array(top_left_corners)

def makeTissueMask(img):
    ''' Makes a tissue mask. Also filters the green / blue pen markings '''

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    # Filter green/blue ink marker annotations
    lower_red = np.array([120,20,180])
    upper_red = np.array([190,220,255])
    tissue_mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Post-process
    tissue_mask = cv2.dilate(tissue_mask, None, iterations=2)
    tissue_mask = cv2.morphologyEx(tissue_mask, cv2.MORPH_CLOSE, None)
    tissue_mask = cv2.medianBlur(tissue_mask, 21)
    
    return tissue_mask

def fillTissueMask(tissue_mask):
    ''' Fills the holes in a tissue mask by filling the corresponding contours '''

    new_tm = tissue_mask.copy()
    contours,_ = cv2.findContours(new_tm,0,cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(new_tm,contours,-1,255,-1)
    
    return new_tm

def estimateWithCircles(arch, radius:int=64):

    arch = arch.squeeze()
    circle_centers=[arch[0]]
    for point in arch:
        if np.all(np.linalg.norm(point-np.array(circle_centers),axis=-1)>radius):
            circle_centers.append(point)
    return np.array(circle_centers)

def getTissuePercentages(tissue_mask, level_offset:int, tile_size:int, top_left_corners):
    ''' Calculate the tissue percentage per each tile. Tissue ~ non 255 on the red-channel '''

    ds_rate = 4**level_offset
    tissue_pcts = np.array([tissue_mask[j//ds_rate:(j+tile_size)//ds_rate,
                                        i//ds_rate:(i+tile_size)//ds_rate].sum()
                            for (i,j) in top_left_corners])/(255*(tile_size/ds_rate)**2)
    
    return tissue_pcts

def padIfNeeded(img, tgt_width:int=128, tgt_height:int=128, border_color:int=255):
    ''' Pad images that need padding (padding on right and bottom)'''
    
    h,w = img.shape[0:2]
    
    if w<tgt_width or h<tgt_height:
        padded = np.ones((tgt_height,tgt_width,3), dtype='uint8')*border_color
        padded[:h,:w] = img
        return padded

    return img

def distributeIntToChunks(available:int, weights):
    '''
        To distribute `available` "seats" based on weights, see
        https://stackoverflow.com/questions/9088403/distributing-integers-using-weights-how-to-calculate
    '''
    
    distributed_amounts = []
    total_weights = sum(weights)
    for weight in weights:
        weight = float(weight)
        p = weight / total_weights
        distributed_amount = round(p * available)
        distributed_amounts.append(distributed_amount)
        total_weights -= weight
        available -= distributed_amount
    return np.int32(distributed_amounts)


# ~~~~~~~~~~~~ Slide class ~~~~~~~~~~~~
class Slide:
    
    def __init__(self,slide_fn, level=2, tile_size=128, mask_fn=None, data_provider=None):
        self.slide_fn = slide_fn
        self.level = level
        self.tile_size = tile_size

        self.img = MultiImage(self.slide_fn)[self.level].copy()
        self.dims = np.array(self.img.shape[:2][::-1])
        self.ds_img = MultiImage(self.slide_fn)[2].copy()
        self.tissue_mask = makeTissueMask(self.ds_img)


        self.mask_fn = mask_fn
        self.data_provider = data_provider
        if not (self.mask_fn==None or self.data_provider==None):
            self.mask = MultiImage(mask_fn)[level].sum(axis=-1)

        self.tile_coords = None


    def getTileCoords(self, num_tiles:int, sampling_method='skeleton', tissue_th:tuple=(0.2, 0.7), seed=None, ):
        ''' 
            Find `num_indices` indices with maximal amount of tissue. `tissue_th`  is the slice of tissue percentage ~(min, max)
            within which we allow the search: sometimes the `max` might not give enough tiles 
            for the mosaic, so we can decrease it gradually until `min` is reached. 
        '''

        assert sampling_method in {'skeleton', 'tissue_pct', 'slic'}
        "`sampling_method` should be one of 'skeleton', 'tissue_pct' or 'slic'"
        
        level_offset = 2 - self.level
        
        if sampling_method == 'skeleton':

            # Determine skeleton from filled tissue mask
            filled_tissue_mask = fillTissueMask(self.tissue_mask.copy())
            filled_tissue_mask = filled_tissue_mask // 255  # needs to be an array of 0s and 1s
            skeleton = skeletonize(filled_tissue_mask, method='lee')
            self.skeleton = np.uint8(np.where(skeleton != 0, 255, 0))

            skeleton = cv2.dilate(self.skeleton, None)
            contours, _ = cv2.findContours(skeleton, 0, cv2.CHAIN_APPROX_NONE)

            # Filter contours based on length
            arch_lens = []
            valid_indices = []
            radius=int( self.tile_size / (4 ** level_offset) )
            for idx, arch in enumerate(contours):
                c = cv2.arcLength(arch, False)
                c = c / 2
                if c < radius / 3:
                    continue

                valid_indices.append(idx)
                arch_lens.append(c)

            if not np.array(valid_indices).size == 0:
                contours = np.array(contours)[valid_indices]

            # Extract points from the accepted contours
            weights = np.array(arch_lens) / np.sum(arch_lens)
            points_per_arch = distributeIntToChunks(num_tiles, weights)  # <- number of points to be extracted
            for idx, arch in enumerate(contours):

                num_indices = points_per_arch[idx]
                output = np.zeros_like(skeleton)
                cv2.drawContours(output, [arch], -1, 1, 1)

                y_, x_ = np.where(output)

                # Simplify the shape by fitting circles
                arch = np.dstack([x_, y_])
                cps = estimateWithCircles(arch, radius)
                cx, cy = cps[..., 0], cps[..., 1]  #

                # Randomly select indices, in case too many; seed if needed
                if len(cx) > num_indices:

                    # Seed if needed
                    if not seed == None:
                        np.random.seed(seed)
                    indices = sorted(np.random.choice(len(cx), points_per_arch[idx], replace=False))
                    np.random.seed(None)  # Return clock seed

                    cx, cy = cx[indices], cy[indices]

                # Append to returnables
                if idx == 0:
                    intermediate_coords = np.dstack([cx, cy])
                else:
                    intermediate_coords = np.hstack([intermediate_coords, np.dstack([cx, cy])])

            # To top-left-corner format
            final_coords = intermediate_coords.squeeze()*4**level_offset
            final_coords = final_coords - np.array( [self.tile_size//2,self.tile_size//2])

            self.tile_coords = final_coords.copy()

        elif sampling_method == 'tissue_pct':
            self.top_left_corners = getTopLeftCorners(self.dims, self.tile_size)
            self.tissue_pcts = getTissuePercentages(self.tissue_mask, level_offset=level_offset,
                                                    tile_size=self.tile_size,
                                                    top_left_corners=self.top_left_corners)

            # Find indices
            tth_min, tth = tissue_th
            while len(np.where(self.tissue_pcts > tth)[0]) < num_tiles:
                if tth <= tth_min:
                    break

                tth -= 0.05

            # Indices
            indices = np.where(self.tissue_pcts > tth)[0]

            # Randomly select indices, in case too many; seed if needed
            if len(indices)>num_tiles:
                if not seed == None:
                    np.random.seed(seed)
                indices = sorted(np.random.choice(indices, num_tiles, replace=False))
                np.random.seed(None)  # Return clock seed

            self.tile_coords = self.top_left_corners[indices].copy()

        elif sampling_method == 'slic':

            # Determine SLIC clusters from filled tissue mask
            filled_tissue_mask = fillTissueMask(self.tissue_mask.copy())
            filled_tissue_mask = filled_tissue_mask // 255  # needs to be an array of 0s and 1s

            segments = seg.slic(self.ds_img, compactness=10, seed_type='nplace', mask=filled_tissue_mask, n_segments=num_tiles,
                                multichannel=True, recompute_seeds=True, enforce_connectivity=True)
            indices = [k for k in np.unique(segments) if not k==-1]

            # Randomly select indices, in case too many; seed if needed
            if len(indices)>num_tiles:
                if not seed == None:
                    np.random.seed(seed)
                indices = sorted(np.random.choice(indices, num_tiles, replace=False))
                np.random.seed(None)  # Return clock seed

            for i in indices:
                contours, _ = cv2.findContours(np.uint8(np.where(segments==i, 255, 0)), 0, 1)
                contours = sorted(contours, key=lambda x: cv2.contourArea(x))[::-1]
                
                M = cv2.moments(contours[0])
                cx = np.int32(M['m10'] / M['m00'])
                cy = np.int32(M['m01'] / M['m00'])

                if i==0:
                    intermediate_coords = np.dstack([cx, cy])
                else:
                    intermediate_coords = np.hstack([intermediate_coords, np.dstack([cx, cy])]) 
                           
            # Append more cluster contours if num_tiles has not been reached
            if len(indices)<num_tiles:
                enough_tiles = False
                for i in indices:
                    contours, _ = cv2.findContours(np.uint8(np.where(segments==i, 255, 0)), 0, 1)
                    contours = sorted(contours, key=lambda x: cv2.contourArea(x))[::-1]
                    
                    for j, cnt in enumerate(contours):
                        # accept a slic cluster contour if it's area is at least 5% of tile area
                        if j!=0 and cv2.contourArea(cnt)>(0.05 * self.tile_size / (4**level_offset))**2:
                            M = cv2.moments(cnt)
                            cx = np.int32(M['m10'] / M['m00'])
                            cy = np.int32(M['m01'] / M['m00'])

                            intermediate_coords = np.hstack([intermediate_coords, np.dstack([cx, cy])])
                            
                            if intermediate_coords.shape[1] == 12:
                                enough_tiles = True
                                break
                    if enough_tiles:
                        break
                            
            # To top-left-corner format
            final_coords = intermediate_coords.squeeze() * 4 ** level_offset
            final_coords = final_coords - np.array([self.tile_size // 2, self.tile_size // 2])
            self.tile_coords = final_coords.copy()

    def getTiles(self, stack:bool=False, sampling_method:str='skeleton', mosaic_grid:tuple=(4,3), output_tile_size:int=128, tissue_th:tuple=(0.1, 0.7), seed:int=None):
        ''' Get tiles from the slide and stack into mosaic if needed '''

        # Solve indices to be used in mosaic
        m, n = mosaic_grid
        self.getTileCoords(num_tiles=n*m, sampling_method=sampling_method, tissue_th=tissue_th, seed=seed)

        # Read regions
        output_img = np.ones([n * m, output_tile_size, output_tile_size, 3], dtype='uint8') * 255

        for idx, coord in enumerate(self.tile_coords):
            x, y = coord
            left, right = np.int32(np.clip([x, x + self.tile_size], 0, self.dims[0]))
            bottom, top = np.int32(np.clip([y, y + self.tile_size], 0, self.dims[1]))

            tile = self.img[bottom:top, left:right].copy()
            tile = padIfNeeded(tile, tgt_width=self.tile_size, tgt_height=self.tile_size)
            tile = cv2.resize(tile, (output_tile_size,) * 2)

            output_img[idx] = np.uint8(tile)

        if len(self.tile_coords)<m*n:
            warnings.warn("Could not find enough unique tiles for the slide"
                          "(tiles: %s/%s, slide: %s" %(len(self.tile_coords), m*n, self.slide_fn))

        # Stack to single array of (m,n) tiles if needed
        if stack:
            output_img = [np.hstack([output_img[i * n + j] for j in range(n)]) for i in range(m)]
            output_img = np.vstack(output_img)

        return np.array(output_img)

    def getTilesCancerStatus(self, stack:bool=False, mosaic_grid:tuple=(4,3)):
        ''' Get tiles from the slide and stack into mosaic if needed '''

        # Solve indices to be used in mosaic
        m, n = mosaic_grid

        # Read regions
        output_img = np.zeros([n * m], dtype='uint8')

        for idx, coord in enumerate(self.tile_coords):
            x, y = coord
            left, right = np.int32(np.clip([x, x + self.tile_size], 0, self.dims[0]))
            bottom, top = np.int32(np.clip([y, y + self.tile_size], 0, self.dims[1]))

            tile = self.mask[bottom:top, left:right].copy()
            tile_cat = tileClassification(tile, self.data_provider)

            output_img[idx] = tile_cat

        # Stack to single array of (m,n) tiles if needed
        if stack:
            output_img = [np.hstack([output_img[i * n + j] for j in range(n)]) for i in range(m)]
            output_img = np.vstack(output_img)

        return np.array(output_img)

    def visualizeCoverage(self, figsize=(16,16)):
        ''' Visualize the coverage of indices on a slide '''

        background = self.ds_img.copy()
        foreground = background.copy()

        level_offset = 2-self.level

        for coord in self.tile_coords:
            x, y = coord
            left, right = np.int32(np.clip([x, x + self.tile_size], 0, self.dims[0])/(4**level_offset))
            bottom, top = np.int32(np.clip([y, y + self.tile_size], 0, self.dims[1])/(4**level_offset))

            foreground[bottom:top, left:right] = (0, 255, 0)

        ## Visualize
        output = cv2.addWeighted(background, 0.7, foreground, 0.3, 0)
        
        plt.figure(figsize=figsize)
        plt.imshow(output)
        plt.show()

# Data

In [ ]:
class PandaDataset(Dataset):
    def __init__(self, path, test, level, 
                 tile_size, sz, mosaic_grid,
                 mean,std,
                 tissue_th=(0.2,0.7), sampling_method='skeleton', seed=123):
        self.path = path
        self.names = list(pd.read_csv(test).image_id)
        self.level = level
        self.sz = sz
        self.tile_size = tile_size
        self.mosaic_grid = mosaic_grid
        self.mean = mean
        self.std = std
        self.tissue_th = tissue_th
        self.sampling_method = sampling_method
        self.seed = seed
        self.N = mosaic_grid[0]*mosaic_grid[1]
    def __len__(self):
        return len(self.names)
    def __getitem__(self, idx):
        name = self.names[idx]
        fn = os.path.join(self.path,name+'.tiff')
        try:
            slide = Slide(fn, level=self.level, tile_size=self.tile_size)

            tiles = slide.getTiles(mosaic_grid=self.mosaic_grid,
                                   stack=False,
                                   sampling_method=self.sampling_method,
                                   output_tile_size=self.sz,
                                   tissue_th=self.tissue_th, seed=self.seed)
        except Exception as e:
            print("Error in tile generation: %s" %e)
            tiles = np.zeros((self.N,self.sz,self.sz,3)).astype(np.float32)
        tiles = tiles.reshape(-1,self.sz,self.sz,3)
        tiles = torch.Tensor(1.0 - tiles/255.0)
        tiles = (tiles - self.mean)/self.std
        return tiles.permute(0,3,1,2), name

## Slide sampling + Dataloader unit test

In [ ]:
TRAIN_DATA = '../input/prostate-cancer-grade-assessment/train_images/'
TRAIN = '../input/prostate-cancer-grade-assessment/train.csv'
dls = []
def to_one(data, sz, mean, std, N):
    img = torch.stack(data,1)
    img = img.view(3,-1,N,sz,sz).permute(0,1,3,2,4).contiguous().view(3,-1,sz*N)
    img = 1.0 - (mean[...,None,None]+img*std[...,None,None])
    return Image(img)

# don't crash if training data is missing
if os.path.exists(TRAIN_DATA):
    
    for i, _ in enumerate(models):
        # load training data and correct labels
        ds = PandaDataset(
            TRAIN_DATA,
            TRAIN,
            level=level[i],
            tile_size=tile_size[i],
            sz=sz[i],
            mosaic_grid=mosaic_grid[i],
            mean=mean[i],
            std=std[i],
            tissue_th=tissue_th,
            sampling_method=sampling_method[i],
            seed=seed
        )
        dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
        dls.append(dl)
        train_df = pd.read_csv(TRAIN).set_index('image_id')
        for (x,y) in dl:
            break
        # display batch tiles
        for img in x:
            display(to_one(list(img), sz[i], mean[i], std[i], N[i]))

## Model unit test

In [ ]:
# quadratic weights
w = np.zeros((6,6))
for i in range(len(w)):
    for j in range(len(w)):
        w[i][j] = float(((i-j)**2)/16)
w

In [ ]:
def ordinalRegs2cat(arr, classes=[0,1,2,3,4,5]):
    #mask = arr == 0
    #return np.clip(np.where(mask.any(1), mask.argmax(1), len(classes)) - 1, classes[0], classes[-1])
    return np.sum(arr,1)

def regr2cat(p):
    for idx,pred in enumerate(p):
        if   pred < coef[0]: p[idx] = 0
        elif pred < coef[1]: p[idx] = 1
        elif pred < coef[2]: p[idx] = 2
        elif pred < coef[3]: p[idx] = 3
        elif pred < coef[4]: p[idx] = 4
        else:                p[idx] = 5
    return p

In [ ]:
%%time
from sklearn.metrics import cohen_kappa_score

RUN_N_TRAIN_SAMPLES = 50

# don't crash if training data is missing
if os.path.exists(TRAIN_DATA):
    names,preds = [],[]
    
    with torch.no_grad():
        for i, samples in tqdm(enumerate(zip(*dls))):
            meta_model_inputs = np.zeros((bs,len(dls)))
            len_models = 0
            for j, (x,y) in enumerate(samples):
                x = x.cuda().float()
                #dihedral TTA
                x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
                  x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
                  x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],1)
                x = x.view(-1,N[j],3,sz[j],sz[j])
                if is_ordinal[j]:
                    p = [tensor(ordinalRegs2cat((torch.sigmoid(model(x)) > 0.5).cpu().numpy())).float().cuda() for model in models[j]]
                elif regr[j]:
                    p = [regr2cat(model(x)) for model in models[j]]
                else:
                    p = [model(x).argmax(-1).float() for model in models[j]]
                
                # TTA averages - keep models separate
                p = torch.stack(p,1).view(bs,len(models[j]), 8, -1) 
                p = p.mean(axis=2)
                for idx,preds_item in enumerate(p):
                    softmax_sum = np.zeros((6), np.float) # number of classes
                    for preds_fold, cm in zip(preds_item, cms[j]):
                        # get class integer
                        if not regr and not is_ordinal:
                            pred_cl = int(preds_fold.argmax(-1))
                        else:
                            if   preds_fold < coef[0]: pred_cl = 0
                            elif preds_fold < coef[1]: pred_cl = 1
                            elif preds_fold < coef[2]: pred_cl = 2
                            elif preds_fold < coef[3]: pred_cl = 3
                            elif preds_fold < coef[4]: pred_cl = 4
                            else:                pred_cl = 5
                
                        cl_probs = cm[:,pred_cl]
                        softmax_probs = cl_probs / np.sum(cl_probs)
                        softmax_sum += softmax_probs
                    meta_model_inputs[idx,j] = np.argmax(softmax_sum)
                del x
            
            ps = np.round(np.average(meta_model_inputs.reshape(bs,-1), axis=1, weights=model_weights))
            
            names.append(y)
            preds.append(torch.tensor(ps))
            # stop unit testing after enough samples has been collected
            if i >= (RUN_N_TRAIN_SAMPLES//bs - 1):
                print("Unit test succeeded.")
                break
    gt_grades = np.array([train_df.loc[pred_ids]['isup_grade'].values for pred_ids in names]).flatten()
    pred_grades = regr2cat(np.array([pred.numpy() for pred in preds]).flatten())
    qwk = cohen_kappa_score(gt_grades, pred_grades, weights="quadratic")
    print("QWK: {0:.4f}".format(qwk))
    assert qwk>0.7, "QWK test gave low scores. Something is probably wrong."
    
    
    names = np.concatenate(names)
    preds = regr2cat(np.array([pred.numpy() for pred in preds]).flatten()).astype(np.int32)
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})
    print(sub_df.head())

# Prediction

In [ ]:
sub_df = pd.read_csv(SAMPLE)
if os.path.exists(DATA):
    dls = []
    
    for i, _ in enumerate(models):
        # load training data and correct labels
        ds = PandaDataset(
            DATA,
            TEST,
            level=level[i],
            tile_size=tile_size[i],
            sz=sz[i],
            mosaic_grid=mosaic_grid[i],
            mean=mean[i],
            std=std[i],
            tissue_th=tissue_th,
            sampling_method=sampling_method[i],
            seed=seed
        )
        dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
        dls.append(dl)
    
    names,preds = [],[]

    with torch.no_grad():
        for i, samples in tqdm(enumerate(zip(*dls))):
            meta_model_inputs = np.zeros((bs,len(dls)))
            len_models = 0
            for j, (x,y) in enumerate(samples):
                x = x.cuda().float()
                #dihedral TTA
                x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),
                  x.transpose(-1,-2),x.transpose(-1,-2).flip(-1),
                  x.transpose(-1,-2).flip(-2),x.transpose(-1,-2).flip(-1,-2)],1)
                x = x.view(-1,N[j],3,sz[j],sz[j])
                if is_ordinal[j]:
                    p = [tensor(ordinalRegs2cat((torch.sigmoid(model(x)) > 0.5).cpu().numpy())).float().cuda() for model in models[j]]
                elif regr[j]:
                    p = [regr2cat(model(x)) for model in models[j]]
                else:
                    p = [model(x).argmax(-1).float() for model in models[j]]
                
                # TTA averages - keep models separate
                p = torch.stack(p,1).view(bs,len(models[j]), 8, -1) 
                p = p.mean(axis=2)
                for idx,preds_item in enumerate(p):
                    softmax_sum = np.zeros((6), np.float) # number of classes
                    for preds_fold, cm in zip(preds_item, cms[j]):
                        # get class integer
                        if not regr and not is_ordinal:
                            pred_cl = int(preds_fold.argmax(-1))
                        else:
                            if   preds_fold < coef[0]: pred_cl = 0
                            elif preds_fold < coef[1]: pred_cl = 1
                            elif preds_fold < coef[2]: pred_cl = 2
                            elif preds_fold < coef[3]: pred_cl = 3
                            elif preds_fold < coef[4]: pred_cl = 4
                            else:                pred_cl = 5
                
                        cl_probs = cm[:,pred_cl]
                        softmax_probs = cl_probs / np.sum(cl_probs)
                        softmax_sum += softmax_probs
                    meta_model_inputs[idx,j] = np.argmax(softmax_sum)
                del x
            
            ps = np.round(np.average(meta_model_inputs.reshape(bs,-1), axis=1, weights=model_weights))
            
            names.append(y)
            preds.append(torch.tensor(ps))
    
    names = np.concatenate(names)
    preds = regr2cat(np.array([pred.numpy() for pred in preds]).flatten()).astype(np.int32)
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})
    sub_df.to_csv('submission.csv', index=False)
    sub_df.head()

In [ ]:
sub_df.to_csv("submission.csv", index=False)
sub_df.head()